In [13]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version


# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [14]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [15]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
Home_sales_lookup = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=',', header=True)


from pyspark.sql.functions import year
Home_sales_lookup.select(year(Home_sales_lookup["date"]))
Home_sales_lookup = Home_sales_lookup.withColumn("year", year(Home_sales_lookup["date"]))
Home_sales_lookup.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|2019-01-30|      2

In [16]:
# 2. Create a temporary view of the DataFrame.
Home_sales_lookup.createOrReplaceTempView('home_sales')


In [17]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
  SELECT
    Year(date) as YEAR,
    ROUND(AVG(price), 2) AS AVERAGE_PRICE
  FROM home_sales
  WHERE Bedrooms = 4
  GROUP BY YEAR(date)
  ORDER BY YEAR DESC
  """

spark.sql(query).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [18]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
  SELECT
    Year(date) as YEAR,
    ROUND(AVG(price), 2) AS AVERAGE_PRICE
  FROM home_sales
  WHERE Bedrooms = 3 AND BATHROOMS = 3
  GROUP BY YEAR(date)
  ORDER BY YEAR DESC
  """

spark.sql(query).show()

+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    292725.69|
|2021|    294211.46|
|2020|    294204.16|
|2019|    287287.82|
+----+-------------+



In [19]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
  SELECT
    Year(date) as YEAR,
    ROUND(AVG(price), 2) AS AVERAGE_PRICE
  FROM home_sales
  WHERE Bedrooms = 3 AND BATHROOMS = 3 AND floors = 2 AND sqft_lot >= 2000
  GROUP BY YEAR(date)
  ORDER BY YEAR DESC
  """

spark.sql(query).show()

+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    287162.18|
|2021|    290319.06|
|2020|    293176.41|
|2019|    282752.72|
+----+-------------+



In [20]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()


query = """
  SELECT
    view,
    ROUND(AVG(price), 2) AS AVERAGE_PRICE
  FROM home_sales
  WHERE price >= 350000
  GROUP BY view
  """

spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|   7|    403005.77|
|  51|    788128.21|
|  15|     404673.3|
|  54|    798684.82|
|  11|    399548.12|
|  29|    397771.65|
|  69|    750537.94|
|  42|     396964.5|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|   3|     398867.6|
|  30|     397862.0|
|  34|    401419.75|
|  59|     791453.0|
|   8|    398592.71|
|  28|    402124.62|
|  22|    402022.68|
|  85|   1056336.74|
|  16|    399586.53|
+----+-------------+
only showing top 20 rows

--- 0.6472697257995605 seconds ---


In [21]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [22]:
# 8. Check if the table is cached.
spark.catalog.isCached("home_sales")

True

In [23]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()


query = """
  SELECT
    view,
    ROUND(AVG(price), 2) AS AVERAGE_PRICE
  FROM home_sales
  WHERE price >= 350000
  GROUP BY view
  """
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|   7|    403005.77|
|  51|    788128.21|
|  15|     404673.3|
|  54|    798684.82|
|  11|    399548.12|
|  29|    397771.65|
|  69|    750537.94|
|  42|     396964.5|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|   3|     398867.6|
|  30|     397862.0|
|  34|    401419.75|
|  59|     791453.0|
|   8|    398592.71|
|  28|    402124.62|
|  22|    402022.68|
|  85|   1056336.74|
|  16|    399586.53|
+----+-------------+
only showing top 20 rows

--- 0.5865192413330078 seconds ---


In [26]:
!pip install parquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.4/643.4 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.9 MB/s eta 0:00:00
  Created wheel for thriftpy2: filename=thriftpy2-0.4.16-cp310-cp310-linux_x86_64.whl size=1078713 sha256=83b2228427a9b6d85d72ab47b426935db07acc6e68ff4337bcff8d20183cc632
  Stored in directory: /root/.cache/pip/wheels/95/f5/1d/fe404692e1c8aaea45220c322d1d0f32c9fd40eb0e2bdd571e
Successfully built thriftpy2


In [29]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
Home_sales_lookup.write.partitionBy("date_built").mode('overwrite').parquet("home_parquet")

In [31]:
# 11. Read the parquet formatted data.
Par_data =spark.read.parquet('home_parquet')

In [32]:
# 12. Create a temporary table for the parquet data.
Par_data.createOrReplaceTempView('Par_data')

In [45]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query = '''(
  SELECT
    view, ROUND(AVG(price), 2) AS AVERAGE_PRICE
  FROM Par_data
  WHERE price >= 350000
  GROUP BY view
)'''
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|   7|    403005.77|
|  51|    788128.21|
|  15|     404673.3|
|  54|    798684.82|
|  11|    399548.12|
|  29|    397771.65|
|  69|    750537.94|
|  42|     396964.5|
|  73|    752861.18|
|  87|    1072285.2|
|  64|    767036.67|
|   3|     398867.6|
|  30|     397862.0|
|  34|    401419.75|
|  59|     791453.0|
|   8|    398592.71|
|  28|    402124.62|
|  22|    402022.68|
|  85|   1056336.74|
|  35|    401934.21|
+----+-------------+
only showing top 20 rows

--- 1.712787389755249 seconds ---


In [46]:
# 14. Uncache the home_sales temporary table.
spark.sql("cache table home_sales")

DataFrame[]

In [47]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales")

True